# Load Data

In [1]:
from pyearth import Earth
from DLtools.Data import instant_data,check_specific_col,station_sel,del_less_col

import numpy as np
import pandas as pd

In [2]:

def to_supervise(data,target,n_out):
    data[target]=data[target].shift(-n_out)
    data = data.astype('float64').dropna()
    X = data.drop([target],axis=1)
    xlabels = list(X.columns)

    X = X.values
    y = data[target].values
    return X,y,xlabels

In [13]:
loading = instant_data()
# df,mode = loading.hourly_instant(),'hour'
df,mode = loading.daily_instant(),'day'

if mode =='hour': n_past,n_future = 24*7,72
elif mode =='day': n_past,n_future = 30,14
else: n_future=None; print('incorrect input')

st = 'CPY012'
target,start_p,stop_p,host_path=station_sel(st,mode)
save_path = '/home/song/Public/Song/Work/Thesis/MAR/'

In [14]:
data = df[start_p:stop_p]
data = del_less_col(data,ratio=.85).interpolate(limit=3000000000,limit_direction='both').astype('float32')
data['Day'] = data.index.dayofyear #add day
data.shape


Before del col are... 305
After... 219


(1520, 220)

In [15]:
# ## Shift target station to future 
n_out = n_future
data[target]=data[target].shift(-n_out)
data = data.astype('float32').dropna()

X = data.drop([target],axis=1)
xlabels = list(X.columns)


# # # REPLACE NAN WITH 0
# # X = X.fillna(0).values
# # y = np.log(data[TARGET].fillna(0).values)
# # where_are_NaNs = np.isnan(y)
# # y[where_are_NaNs] = 0
X=X.values
y=data[target].values

# MARS feature selection (from all stations)

In [16]:
#Fit an Earth model
criteria = ('rss', 'gcv', 'nb_subsets')
model = Earth(enable_pruning = True,
            #   max_degree=3,
            #   max_terms=10,
              minspan_alpha=.5,
              feature_importance_type=criteria,
              verbose=True)
model.fit(X,y,xlabels=xlabels)


Beginning forward pass
---------------------------------------------------------------
iter  parent  var  knot  mse       terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.491022  1      0.492  0.000  0.000  
1     0       98   215   0.105363  3      0.106  0.785  0.784  
2     0       107  1272  0.080635  5      0.082  0.836  0.834  
3     0       105  663   0.070864  7      0.072  0.856  0.853  
4     0       117  156   0.064636  9      0.066  0.868  0.865  
5     0       195  865   0.059703  11     0.062  0.878  0.874  
6     0       100  1000  0.056364  13     0.059  0.885  0.880  
7     0       99   209   0.053254  15     0.056  0.892  0.886  
8     0       111  1304  0.050987  17     0.054  0.896  0.890  
9     0       146  446   0.048446  19     0.052  0.901  0.895  
10    0       97   1201  0.046229  21     0.050  0.906  0.899  
11    0       90   1338  0.044293  23     0.048  0.910  0.903  
12    0       24 

Earth(feature_importance_type=('rss', 'gcv', 'nb_subsets'), minspan_alpha=0.5,
      verbose=True)

In [62]:
# #Print the model
# print(model.trace())
# print(model.summary())

* Feature Importance

In [9]:
print(model.summary_feature_importances(sort_by='rss')[:700])

                                   rss    gcv    nb_subsets
CPY006_wl                          0.83   0.84   0.05          
NAN013_wl                          0.04   0.04   0.03          
KWN002_wl                          0.02   0.02   0.03          
DIV006_wl                          0.02   0.01   0.10          
Dam_SK_Useable_WaterVol            0.01   0.01   0.03          
CPY009_wl                          0.01   0.01   0.03          
CPY008_wl                          0.01   0.01   0.03          
CPY016_wl                          0.01   0.01   0.03          
CPY004_wl                          0.01   0.01   0.03          
YOM008_wl                          0.01   0.01   0.03          



In [5]:
def toDF(rank):
    name,rss,gcv,nb_sub= list(),list(),list(),list()
    for i in range(len(rank)):
        if i%4==0:
            name.append(rank[i])
        elif i%4==1:
            rss.append(float(rank[i]))
        elif i%4==2:
            gcv.append(float(rank[i]))
        elif i%4==3:
            nb_sub.append(float(rank[i]))
    data = {'feature':name,
    'rss':rss,
    'gcv':gcv,
    'nb_subset':nb_sub}
    score = pd.DataFrame(data)
    return score



In [7]:
nbsub = model.summary_feature_importances(sort_by='nb_subsets')[:2000].split()[3:83]
gcv = model.summary_feature_importances(sort_by='gcv')[:2000].split()[3:83]
rss = model.summary_feature_importances(sort_by='rss')[:2000].split()[3:83]

rss = toDF(rss)
gcv = toDF(gcv)
nbsub = toDF(nbsub)


NameError: name 'model' is not defined

In [6]:
thrd = 0.05
rss=rss.loc[rss['rss']>thrd]
gcv=gcv.loc[gcv['gcv']>thrd]
nbsub=nbsub.loc[nbsub['nb_subset']>thrd]

NameError: name 'rss' is not defined

In [45]:
#From top 20 rss,gcv,nbsub
top20 = pd.concat([rss,gcv,nbsub],ignore_index=True).drop_duplicates('feature')
top20

,feature,rss,gcv,nb_subset
0,CPY006_wl,0.83,0.84,0.05
2,DIV006_wl,0.02,0.01,0.10


In [39]:
if mode =='hour': top20.to_csv('MAR/[CPY012]featurelist_MAR_hourly.csv',index=False)
elif mode =='day': top20.to_csv('MAR/[CPY012]featurelist_MAR_daily.csv',index=False)
else: print('incorrect input')

# Loop

In [17]:
conclude_df=pd.DataFrame()
for n_out in range(1,n_future):
    X,y,xlabels = to_supervise(data,target,n_out)
    criteria = ('rss', 'gcv', 'nb_subsets')
    model = Earth(enable_pruning = True,
                #   max_degree=3,
                #  max_terms=20,
                minspan_alpha=.5,
                feature_importance_type=criteria,
                verbose=True)
    model.fit(X,y,xlabels=xlabels)
    nbsub = model.summary_feature_importances(sort_by='nb_subsets')[:2000].split()[3:83]
    gcv = model.summary_feature_importances(sort_by='gcv')[:2000].split()[3:83]
    rss = model.summary_feature_importances(sort_by='rss')[:2000].split()[3:83]
    
    rss,gcv,nbsub = toDF(rss),toDF(gcv),toDF(nbsub)
    thrd = 0.1
    rss=rss.loc[rss['rss']>thrd]
    gcv=gcv.loc[gcv['gcv']>thrd]
    nbsub=nbsub.loc[nbsub['nb_subset']>thrd]

    top20=pd.concat([rss,gcv,nbsub],ignore_index=True)
    top20 = pd.concat([rss,gcv,nbsub],ignore_index=True).drop_duplicates('feature')
    top20['timestep'] = n_out

    #ADDED combine all result
    conclude_df = pd.concat([conclude_df,top20],ignore_index=True)
    # conclude_df.to_csv('/home/song/Public/Song/Work/Thesis/MAR/[CPY012]featurelist_MAR_hourly.csv',index=False)

   terms  gcv    rsq    grsq   
---------------------------------------------------------------
0     -       -    -     0.509161  1      0.510  0.000  0.000  
1     0       179  809   0.261586  3      0.264  0.486  0.483  
2     0       182  576   0.185852  5      0.189  0.635  0.630  
3     0       184  206   0.149469  7      0.153  0.706  0.700  
4     0       207  266   0.112980  9      0.116  0.778  0.772  
5     0       92   1283  0.079554  11     0.083  0.844  0.838  
6     0       199  1110  0.071512  13     0.075  0.860  0.854  
7     0       123  1300  0.062076  15     0.065  0.878  0.872  
8     0       130  223   0.057608  17     0.061  0.887  0.880  
9     0       147  1029  0.054550  19     0.058  0.893  0.886  
10    0       134  1347  0.051931  21     0.056  0.898  0.891  
11    0       124  1102  0.049332  23     0.053  0.903  0.895  
12    0       99   1260  0.046774  25     0.051  0.908  0.900  
13    0       117  1198  0.043978  27     0.048  0.914  0.905  
14    0 

In [18]:
final = conclude_df.drop_duplicates('feature')
if mode =='hour': 
    final.to_csv('MAR/[CPY012]featurelist_MAR_hourly.csv',index=False)
    conclude_df.to_csv('MAR/[CPY012]featurelist_MAR_hourly_Fullreport.csv',index=False)
elif mode =='day': 
    final.to_csv('MAR/[CPY012]featurelist_MAR_daily.csv',index=False)
    conclude_df.to_csv('MAR/[CPY012]featurelist_MAR_daily_Fullreport.csv',index=False)
else: print('incorrect input')

In [21]:
conclude_df.loc[conclude_df['timestep']<=4]

,feature,rss,gcv,nb_subset,timestep
0,CPY006_wl,0.81,0.82,0.03,1
1,CPY006_wl,0.67,0.67,0.03,2
2,CPY006_wl,0.70,0.71,0.07,3
3,CPY006_wl,0.67,0.68,0.02,4
4,DIV002_wl,0.12,0.12,0.02,4
